In [1]:
import sys
sys.path.append('..')

In [2]:
# Load SOAR data and filter for target models, add score column

from datasets import load_dataset

ds = load_dataset("julien31/soar_arc_train_5M", columns=['task_id', 'correct_train_input', 'correct_test_input', 'model'])
df = ds['train'].to_pandas()

# Filter for target models and add score
models_to_keep = ['Mistral-Large-Instruct-2407', 'Qwen2.5-72B-Instruct']
df_filtered = df[df['model'].isin(models_to_keep)].copy()
df_filtered['score'] = df_filtered.apply(
    lambda row: (sum(row['correct_train_input']) + sum(row['correct_test_input'])) / 
                (len(row['correct_train_input']) + len(row['correct_test_input'])), 
    axis=1
)

print(f"✅ Loaded SOAR data: {len(df):,} total rows, {len(df_filtered):,} filtered rows")
print(f"Target models: {models_to_keep}")

✅ Loaded SOAR data: 4,926,487 total rows, 1,734,164 filtered rows
Target models: ['Mistral-Large-Instruct-2407', 'Qwen2.5-72B-Instruct']


In [3]:
# Get ARC training task IDs

from task_loader import TaskLoader

loader = TaskLoader(data_root="../../data")
training_path = loader.data_root / "arc-agi-1" / "training"
all_training_task_ids = [f.stem for f in training_path.glob("*.json")]

print(f"✅ Found {len(all_training_task_ids)} ARC training tasks")
print(f"TaskLoader initialized with data root: {loader.data_root}")

✅ Found 400 ARC training tasks
TaskLoader initialized with data root: ../../data


In [4]:
# Select top/bottom 50 per task and preserve original indices

import pandas as pd

df_filtered_with_idx = df_filtered.reset_index()  # Preserve original row indices
selected_indices = []

for task_id in all_training_task_ids:
    task_rows = df_filtered_with_idx[df_filtered_with_idx['task_id'] == task_id]
    if len(task_rows) == 0:
        continue
    
    task_rows_sorted = task_rows.sort_values('score', ascending=True)
    bottom_50 = task_rows_sorted.head(50)
    top_50 = task_rows_sorted.tail(50)
    selected_rows = pd.concat([bottom_50, top_50])
    selected_indices.extend(selected_rows['index'].tolist())

print(f"✅ Selected {len(selected_indices)} SOAR rows from {len(all_training_task_ids)} ARC tasks")
print(f"Average samples per task: {len(selected_indices) / len(all_training_task_ids):.1f}")

✅ Selected 40000 SOAR rows from 400 ARC tasks
Average samples per task: 100.0


In [5]:
# Efficiently load full SOAR data for selected rows only

ds_full = load_dataset("julien31/soar_arc_train_5M")
selected_dataset = ds_full['train'].select(selected_indices)
soar_df = selected_dataset.to_pandas()

# Add score column to full dataset
soar_df['score'] = soar_df.apply(
    lambda row: (sum(row['correct_train_input']) + sum(row['correct_test_input'])) / 
                (len(row['correct_train_input']) + len(row['correct_test_input'])), 
    axis=1
)

print(f"✅ Loaded full SOAR data: {len(soar_df):,} rows")
print(f"Columns: {list(soar_df.columns)[:5]}... ({len(soar_df.columns)} total)")

✅ Loaded full SOAR data: 40,000 rows
Columns: ['code', 'correct_train_input', 'predicted_train_output', 'correct_test_input', 'predicted_test_output']... (9 total)


In [6]:
# Load corresponding ARC task data
print("Loading corresponding ARC task data...")

arc_tasks = {}

for task_id in soar_df['task_id'].unique():
    task_data = loader.load_task(task_id, "arc-agi-1")
    arc_tasks[task_id] = task_data

print(f"✅ Loaded {len(arc_tasks)} ARC tasks")

Loading corresponding ARC task data...
✅ Loaded 400 ARC tasks


In [7]:
# Merge SOAR and ARC data

def add_arc_data(row):
    task_id = row['task_id']
    if task_id in arc_tasks:
        task_data = arc_tasks[task_id]
        row['arc_train_examples'] = task_data.get('train', [])
        row['arc_test_examples'] = task_data.get('test', [])
        row['num_train_examples'] = len(task_data.get('train', []))
        row['num_test_examples'] = len(task_data.get('test', []))
    else:
        row['arc_train_examples'] = []
        row['arc_test_examples'] = []
        row['num_train_examples'] = 0
        row['num_test_examples'] = 0
    return row

final_dataset = soar_df.apply(add_arc_data, axis=1)

print(f"Rows: {len(final_dataset):,}")
print(f"Unique ARC tasks: {final_dataset['task_id'].nunique()}")
print(f"Models: {final_dataset['model'].value_counts().to_dict()}")
print(f"Mean score: {final_dataset['score'].mean():.3f}")

print(f"\nColumns available:")
for col in final_dataset.columns:
    print(f"  - {col}")

Rows: 40,000
Unique ARC tasks: 400
Models: {'Qwen2.5-72B-Instruct': 25635, 'Mistral-Large-Instruct-2407': 14365}
Mean score: 0.352

Columns available:
  - code
  - correct_train_input
  - predicted_train_output
  - correct_test_input
  - predicted_test_output
  - task_id
  - model
  - generation
  - score
  - arc_train_examples
  - arc_test_examples
  - num_train_examples
  - num_test_examples


In [8]:
# Verify dataset integrity by running code on ARC test examples

import random
from tqdm import tqdm
import sys
sys.path.append('..')
from scoring import ProgramExecutor, GridScorer

print("Verifying dataset integrity...")
print("Testing that running the code on ARC test examples produces the same results as recorded in the dataset")

# Sample 100 random rows
sample_size = 100
sample_indices = random.sample(range(len(final_dataset)), min(sample_size, len(final_dataset)))
sample_data = final_dataset.iloc[sample_indices]

executor = ProgramExecutor(timeout=1.0)  # 1 second timeout
scorer = GridScorer()

verification_failures = []
total_test_cases = 0

for idx, row in tqdm(sample_data.iterrows(), total=len(sample_data), desc="Verifying rows"):
    task_id = row['task_id']
    code = row['code']
    arc_test_examples = row['arc_test_examples']
    recorded_correct_test = row['correct_test_input']
    
    if not arc_test_examples:
        print(f"WARNING: No ARC test examples for task {task_id}")
        continue
    
    # Run the code on each test example
    for test_idx, test_example in enumerate(arc_test_examples):
        if test_idx >= len(recorded_correct_test):
            print(f"WARNING: More ARC test examples than recorded results for task {task_id}")
            break
            
        test_input = test_example['input']
        expected_output = test_example['output']
        recorded_correct = recorded_correct_test[test_idx]
        
        # Execute the program
        predicted_output, error, timed_out = executor.execute_program(code, test_input)
        
        if timed_out:
            actual_correct = False
            failure_reason = "timeout"
        elif error or predicted_output is None:
            actual_correct = False
            failure_reason = f"error: {error}"
        else:
            # Score the prediction
            score_result = scorer.score_grid(predicted_output, expected_output)
            actual_correct = score_result['correct']
            failure_reason = "execution_successful"
        
        total_test_cases += 1
        
        # Check if actual result matches recorded result
        if actual_correct != recorded_correct:
            failure_info = {
                'row_idx': idx,
                'task_id': task_id,
                'test_idx': test_idx,
                'recorded_correct': recorded_correct,
                'actual_correct': actual_correct,
                'failure_reason': failure_reason,
                'error': error if error else None
            }
            verification_failures.append(failure_info)
            
            print(f"\n❌ VERIFICATION FAILURE:")
            print(f"   Task: {task_id}, Test case: {test_idx}")
            print(f"   Recorded correct: {recorded_correct}")
            print(f"   Actual correct: {actual_correct}")
            print(f"   Reason: {failure_reason}")
            
            # Exit immediately on first failure as requested
            print(f"\n🛑 Exiting immediately due to verification failure!")
            print(f"Total test cases checked: {total_test_cases}")
            break
    
    # Exit outer loop if we had a failure
    if verification_failures:
        break

if not verification_failures:
    print(f"\n✅ VERIFICATION SUCCESSFUL!")
    print(f"All {total_test_cases} test cases from {len(sample_data)} sampled rows match recorded results")
    print(f"Dataset integrity confirmed - the code executions produce the expected boolean outcomes")
else:
    print(f"\n❌ VERIFICATION FAILED!")
    print(f"Found {len(verification_failures)} mismatches in {total_test_cases} test cases")
    print(f"Dataset may have integrity issues")

Verifying dataset integrity...
Testing that running the code on ARC test examples produces the same results as recorded in the dataset


Verifying rows: 100%|██████████| 100/100 [00:06<00:00, 15.76it/s]


✅ VERIFICATION SUCCESSFUL!
All 103 test cases from 100 sampled rows match recorded results
Dataset integrity confirmed - the code executions produce the expected boolean outcomes


In [9]:
# Setup for generating SFT training examples

import json
from datetime import datetime
from pathlib import Path
import random

from generate_training_data import create_prompt_for_task

def create_training_example_from_soar_row(row):
    """Create a training example from a SOAR dataset row"""
    task_id = row['task_id']
    code = row['code']
    arc_train_examples = row['arc_train_examples']
    arc_test_examples = row['arc_test_examples']
    
    # Create task data structure
    task_data = {
        'train': arc_train_examples,
        'test': arc_test_examples
    }
    
    # Create system message
    system_message = {
        "role": "system", 
        "content": "You are an expert at solving abstract reasoning puzzles. Write clean, efficient Python code."
    }
    
    # Create user message with the prompt
    user_message = {
        "role": "user",
        "content": create_prompt_for_task(task_data)
    }
    
    # Create assistant message with just the program
    assistant_message = {
        "role": "assistant",
        "content": f"Final answer:\n```python\n{code}\n```"
    }
    
    # Create the training example
    training_example = {
        "messages": [system_message, user_message, assistant_message],
        "task_id": task_id,  # Add metadata for tracking
        "score": row['score'],
        "model": row['model']
    }
    
    return training_example

In [10]:
# Setup directories and filter valid data
print("Creating SFT training data from verified SOAR dataset...")

# Create training_data directory if it doesn't exist
training_data_dir = Path("../training_data")
training_data_dir.mkdir(exist_ok=True)

# Filter out rows without ARC examples
valid_rows = final_dataset[
    (final_dataset['num_train_examples'] > 0) & 
    (final_dataset['num_test_examples'] > 0)
].copy()

print(f"Valid rows with ARC examples: {len(valid_rows):,} out of {len(final_dataset):,}")
print(f"✅ Training data directory ready: {training_data_dir}")

Creating SFT training data from verified SOAR dataset...
Valid rows with ARC examples: 40,000 out of 40,000
✅ Training data directory ready: ../training_data


In [11]:
# Create training examples from SOAR data

training_examples = []
for idx, row in valid_rows.iterrows():
    try:
        training_example = create_training_example_from_soar_row(row)
        training_examples.append(training_example)
    except Exception as e:
        print(f"Error creating training example for task {row['task_id']}: {e}")

print(f"✅ Created {len(training_examples):,} training examples")

✅ Created 40,000 training examples


In [12]:
# Split into train/validation sets (90/10 split, balanced by task)

# Set random seed for reproducible splits
random.seed(42)

# Sort by score and split
training_examples_df = pd.DataFrame(training_examples)
training_examples_df = training_examples_df.sort_values('score')

# Group by task to ensure we don't split examples from the same task
task_groups = training_examples_df.groupby('task_id')
task_ids = list(task_groups.groups.keys())
random.shuffle(task_ids)

# 10% for validation (or max 500 examples)
val_size = min(int(len(task_ids) * 0.1), 500 // 2)  # Approximate, will adjust based on examples per task
val_task_ids = task_ids[:val_size]
train_task_ids = task_ids[val_size:]

# Split examples
train_examples = training_examples_df[training_examples_df['task_id'].isin(train_task_ids)].to_dict('records')
val_examples = training_examples_df[training_examples_df['task_id'].isin(val_task_ids)].to_dict('records')

print(f"✅ Split: {len(train_examples)} training, {len(val_examples)} validation")

✅ Split: 36000 training, 4000 validation


In [13]:
# Clean metadata and prepare files for saving

# Remove metadata before saving
for examples in [train_examples, val_examples]:
    for example in examples:
        example.pop('task_id', None)
        example.pop('score', None)
        example.pop('model', None)

train_file = training_data_dir / f"soar_sft_train.jsonl"
val_file = training_data_dir / f"soar_sft_val.jsonl"

# Save training data
with open(train_file, 'w') as f:
    for example in train_examples:
        f.write(json.dumps(example) + '\n')

# Save validation data
with open(val_file, 'w') as f:
    for example in val_examples:
        f.write(json.dumps(example) + '\n')

print(f"✅ SFT datasets created successfully!")
print(f"Training data: {train_file} ({len(train_examples)} examples)")
print(f"Validation data: {val_file} ({len(val_examples)} examples)")

✅ SFT datasets created successfully!
Training data: ../training_data/soar_sft_train.jsonl (36000 examples)
Validation data: ../training_data/soar_sft_val.jsonl (4000 examples)


In [14]:
# Display dataset statistics and summary

# Recreate metadata for statistics (without modifying the saved files)
train_df = pd.DataFrame([{
    'task_id': row['task_id'], 
    'score': row['score'], 
    'model': row['model']
} for row in training_examples_df[training_examples_df['task_id'].isin(train_task_ids)].to_dict('records')])

val_df = pd.DataFrame([{
    'task_id': row['task_id'], 
    'score': row['score'], 
    'model': row['model']
} for row in training_examples_df[training_examples_df['task_id'].isin(val_task_ids)].to_dict('records')])

print(f"\n📊 Dataset statistics:")
print(f"Training set:")
print(f"  - Tasks: {train_df['task_id'].nunique()}")
print(f"  - Score range: {train_df['score'].min():.3f} - {train_df['score'].max():.3f}")
print(f"  - Mean score: {train_df['score'].mean():.3f}")
print(f"  - Models: {train_df['model'].value_counts().to_dict()}")

print(f"Validation set:")
print(f"  - Tasks: {val_df['task_id'].nunique()}")
print(f"  - Score range: {val_df['score'].min():.3f} - {val_df['score'].max():.3f}")
print(f"  - Mean score: {val_df['score'].mean():.3f}")
print(f"  - Models: {val_df['model'].value_counts().to_dict()}")

print(f"\n🎯 Files ready for SFT training!")
print(f"Example training message structure:")
if train_examples:
    # Reload one example to check structure
    with open(train_file, 'r') as f:
        example = json.loads(f.readline())
    print(f"  - System: {len(example['messages'][0]['content'])} chars")
    print(f"  - User: {len(example['messages'][1]['content'])} chars") 
    print(f"  - Assistant: {len(example['messages'][2]['content'])} chars")


📊 Dataset statistics:
Training set:
  - Tasks: 360
  - Score range: 0.000 - 1.000
  - Mean score: 0.349
  - Models: {'Qwen2.5-72B-Instruct': 23074, 'Mistral-Large-Instruct-2407': 12926}
Validation set:
  - Tasks: 40
  - Score range: 0.000 - 1.000
  - Mean score: 0.381
  - Models: {'Qwen2.5-72B-Instruct': 2561, 'Mistral-Large-Instruct-2407': 1439}

🎯 Files ready for SFT training!
Example training message structure:
  - System: 92 chars
  - User: 2828 chars
  - Assistant: 680 chars


In [15]:
# Show examples of the generated training data
print("📋 Example training data structure:")
print("=" * 60)

# Load and show a sample from the training file
with open(train_file, 'r') as f:
    sample_line = f.readline()
    sample_data = json.loads(sample_line)

print("System message:")
print("-" * 40)
print(sample_data['messages'][0]['content'])

print("\nUser message (first 500 chars):")
print("-" * 40)
print(sample_data['messages'][1]['content'][:500] + "...")

print("\nAssistant message (first 300 chars):")
print("-" * 40)
print(sample_data['messages'][2]['content'][:300] + "...")

📋 Example training data structure:
System message:
----------------------------------------
You are an expert at solving abstract reasoning puzzles. Write clean, efficient Python code.

User message (first 500 chars):
----------------------------------------
You are solving an ARC (Abstraction and Reasoning Corpus) task. 
I will show you training examples with input and output grids, plus a test input grid. Your task is to:

1. **Analyze the training examples** to discover patterns that map input grids to output grids
2. **Write a Python program** that implements your best understanding of the transformation  
3. **DO NOT predict or generate the test output** - your job is only to write the transformation program
4. **Attempt a solution** - even if ...

Assistant message (first 300 chars):
----------------------------------------
Final answer:
```python
def transform(grid_lst: list[list[int]]) -> list[list[int]]:
    grid = grid_lst.copy()
    n = len(grid)
    m = len(grid[0])
    bor

In [ ]:
# Upload the SOAR SFT dataset to Hugging Face Hub

from datasets import Dataset, DatasetDict
from huggingface_hub import HfApi, login
import json

# Login to Hugging Face (if not already logged in)
try:
    api = HfApi()
    user = api.whoami()
    print(f"Already logged in as: {user['name']}")
except Exception:
    print("Please log in to Hugging Face Hub:")
    # login()

# Load the JSONL files back into datasets
def load_jsonl_to_dataset(file_path):
    """Load a JSONL file into a HuggingFace Dataset"""
    with open(file_path, 'r') as f:
        data = [json.loads(line) for line in f]
    return Dataset.from_list(data)

print("Loading JSONL files into HuggingFace Datasets...")
train_dataset = load_jsonl_to_dataset(train_file)
val_dataset = load_jsonl_to_dataset(val_file)

# Create a DatasetDict
soar_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})
# Create dataset info
dataset_description = """
# SOAR ARC-AGI SFT Dataset

This dataset contains verified examples from the SOAR dataset, specifically curated for supervised fine-tuning on ARC (Abstraction and Reasoning Corpus) tasks.

## Dataset Details

- **Total Examples**: 40,000 (36,000 train, 4,000 validation)
- **Source Models**: Mistral-Large-Instruct-2407, Qwen2.5-72B-Instruct
- **Task Type**: Abstract reasoning puzzles with Python code solutions
- **Quality**: Verified dataset integrity - all code executions match recorded outcomes
- **Format**: Standard chat format with system/user/assistant messages

## Dataset Structure

Each example contains:
- `messages`: List of chat messages (system, user, assistant)
  - System: Role definition for ARC reasoning
  - User: Full ARC task prompt with training examples and test input
  - Assistant: Python code solution in required format

## Usage

```python
from datasets import load_dataset

dataset = load_dataset("Trelis/soar-arc-sft-...")
print(dataset)
```

## Citation

Based on the SOAR dataset: https://huggingface.co/datasets/julien31/soar_arc_train_5M

Generated and verified using the ARC-AGI 2025 pipeline.
"""

# Set up the repository name
repo_name = f"Trelis/soar-arc-sft-{datetime.now().strftime('%Y-%m-%d_%H%M')}"

print(f"\nUploading dataset to Hugging Face Hub: {repo_name}")
print("This may take a few minutes...")

try:
    # Push to hub with description and tags
    soar_dataset.push_to_hub(
        repo_name,
        private=True,  # Set to True if you want it private initially
        commit_message="Add verified SOAR ARC-AGI SFT dataset",
    )
    
    # Update the dataset card
    print("Updating dataset card...")
    api.upload_file(
        path_or_fileobj=dataset_description.encode(),
        path_in_repo="README.md",
        repo_id=repo_name,
        repo_type="dataset",
        commit_message="Add dataset description"
    )
    
    print(f"✅ Successfully uploaded dataset to: https://huggingface.co/datasets/{repo_name}")
    print(f"🎯 Ready to use in training scripts with: load_dataset('{repo_name}')")
    
except Exception as e:
    print(f"❌ Error uploading dataset: {e}")
    print("You may need to:")
    print("1. Ensure you're logged in with `huggingface-cli login`")
    print("2. Have write permissions to the Trelis organization")
    print("3. Or change the repo_name to your personal account")

Already logged in as: l3wish
Loading JSONL files into HuggingFace Datasets...
Dataset loaded:
- Train: 36,000 examples
- Validation: 4,000 examples
- Features: ['messages']

Uploading dataset to Hugging Face Hub: Trelis/soar-arc-sft-2025-07-21_01
This may take a few minutes...


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/36 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Updating dataset card...


/home/lewis/code/trelis-arc/llm-python/.venv/lib/python3.12/site-packages/huggingface_hub/hf_api.py:9692: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


✅ Successfully uploaded dataset to: https://huggingface.co/datasets/Trelis/soar-arc-sft-2025-07-21_01
🎯 Ready to use in training scripts with: load_dataset('Trelis/soar-arc-sft-2025-07-21_01')
